In [ ]:
! git clone https://github.com/NVlabs/stylegan
%cd stylegan/

In [15]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib

#----------------------------------------------------------------------------
# Helpers for loading and using pre-trained generators.

url_ffhq        = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
url_celebahq    = 'https://drive.google.com/uc?id=1MGqJl28pN4t7SAtSrPdSRJSQJqahkzUf' # karras2019stylegan-celebahq-1024x1024.pkl
url_bedrooms    = 'https://drive.google.com/uc?id=1MOSKeGF0FJcivpBI7s63V9YHloUTORiF' # karras2019stylegan-bedrooms-256x256.pkl
url_cars        = 'https://drive.google.com/uc?id=1MJ6iCfNtMIRicihwRorsM3b7mmtmK9c3' # karras2019stylegan-cars-512x384.pkl
url_cats        = 'https://drive.google.com/uc?id=1MQywl0FNt6lHu8E_EUqnRbviagS7fbiJ' # karras2019stylegan-cats-256x256.pkl

synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=8)

_Gs_cache = dict()

def load_Gs(url):
    if url not in _Gs_cache:
        with dnnlib.util.open_url(url, cache_dir=os.path.join(os.getcwd(), "cache_dir")) as f:
            _G, _D, Gs = pickle.load(f)
        _Gs_cache[url] = Gs
    return _Gs_cache[url]


In [16]:
# Style mixing - mix n_mix_layer of mix n_mix_layer of mixing image to reference image.
from itertools import chain

tflib.init_tf()
result_dir = os.path.join(os.getcwd(), "result_dir")
os.makedirs(result_dir, exist_ok=True)

png = os.path.join(result_dir, 'figure03-style-mixing.png')
Gs = load_Gs(url_ffhq)
w = 1024
h = 1024
style_ranges = [range(16,18), range(4, 6), range(2, 4), range(0, 2)] # range(14, 16), range(12, 14) , range(10, 12), range(8, 10), range(6, 8),
master_seeds = [701,687,615,2268,888,829,1898,1733,1614,845,1967,1555,1157,1012,91,388,201,500]
src_seeds = [701,687,615,2268,888,829,1898,1733,1614,1967,1555,1157,1012,91,388,201,500] 
dst_seeds = [master_seeds[9]] * len(style_ranges)

src_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in src_seeds)
dst_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in dst_seeds)
src_dlatents = Gs.components.mapping.run(src_latents, None) # [seed, layer, component]
dst_dlatents = Gs.components.mapping.run(dst_latents, None) # [seed, layer, component]
src_images = Gs.components.synthesis.run(src_dlatents, randomize_noise=False, **synthesis_kwargs)
dst_images = Gs.components.synthesis.run(dst_dlatents, randomize_noise=False, **synthesis_kwargs)

canvas = PIL.Image.new('RGB', (w * (len(src_seeds) + 1), h * (len(dst_seeds) + 1)), 'white')
for col, src_image in enumerate(list(src_images)):
    canvas.paste(PIL.Image.fromarray(src_image, 'RGB'), ((col + 1) * w, 0))
for row, dst_image in enumerate(list(dst_images)):
    canvas.paste(PIL.Image.fromarray(dst_image, 'RGB'), (0, (row + 1) * h))
    row_dlatents = np.stack([dst_dlatents[row]] * len(src_seeds))
    row_dlatents[:, style_ranges[row]] = src_dlatents[:, style_ranges[row]]
    row_images = Gs.components.synthesis.run(row_dlatents, randomize_noise=False, **synthesis_kwargs)
    for col, image in enumerate(list(row_images)):
        canvas.paste(PIL.Image.fromarray(image, 'RGB'), ((col + 1) * w, (row + 1) * h))
canvas.save(png)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Users\57luu\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
C:\Users\57luu\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


InvalidArgumentError: Cannot assign a device for operation G_mapping_1/_Run/G_mapping/latents_in: node G_mapping_1/_Run/G_mapping/latents_in (defined at C:\Users\57luu\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1748)  was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device.
	 [[G_mapping_1/_Run/G_mapping/latents_in]]

In [0]:
import matplotlib.pyplot as plt
image_mix_style = plt.imread(png)
plt.figure(figsize=(25,25))
plt.imshow(image_mix_style)
ax_current = plt.gca()
ax_current.axes.get_xaxis().set_visible(False)
ax_current.axes.get_yaxis().set_visible(False)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
# Download directly (works for small files)
from google.colab import files
files.download(png) 

In [0]:
# Download to Drive (works for big files)
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
# Create & upload a file.
file_name = ''.join([str(style_ranges).strip('[]'), ".png"])
uploaded = drive.CreateFile({'title': file_name})
uploaded.SetContentFile(png)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1-PvZinVoq4xlCUxvh4pYMNNVOwIs7EpJ


In [0]:
print(str(style_ranges).strip('[]'))

range(16, 18), range(14, 16), range(12, 14), range(10, 12), range(8, 10), range(6, 8), range(4, 6), range(2, 4), range(0, 2)


In [0]:
# Style mixing - mix n_mix_layer of reference image to mixing image.
from itertools import chain

tflib.init_tf()
os.makedirs(config.result_dir, exist_ok=True)

png = os.path.join(config.result_dir, 'figure03-style-mixing.png')
Gs = load_Gs(url_ffhq)
w = 1024
h = 1024
n_mix_layer = 9
style_ranges = [range(i, i+n_mix_layer) for i in range(0, 18-n_mix_layer+1)]
#for i in range(3):
#  style_ranges.insert(0, style_ranges[0])
#mix_ranges = [range(len(master_seeds)-n_mix_layer,len(master_seeds))] * len(dst_seeds)
#style_ranges = [list(chain(range(0, mix_range[0]), range(mix_range[-1]+1, 18))) for mix_range in mix_ranges]
src_seeds = [639,701,687,615,2268,888,829,1898,1733,1614,845,1967,1555]
dst_seeds = [639,701,687,615,2268,888,829,1898,1733,1614]

src_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in src_seeds)
dst_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in dst_seeds)
src_dlatents = Gs.components.mapping.run(src_latents, None) # [seed, layer, component]
dst_dlatents = Gs.components.mapping.run(dst_latents, None) # [seed, layer, component]
src_images = Gs.components.synthesis.run(src_dlatents, randomize_noise=False, **synthesis_kwargs)
dst_images = Gs.components.synthesis.run(dst_dlatents, randomize_noise=False, **synthesis_kwargs)

canvas = PIL.Image.new('RGB', (w * (len(dst_seeds) + 1), h * (len(src_seeds) + 1)), 'white')
for row, src_image in enumerate(list(src_images)):
    canvas.paste(PIL.Image.fromarray(src_image, 'RGB'), (0, (row + 1) * h))
for col, dst_image in enumerate(list(dst_images)):
    canvas.paste(PIL.Image.fromarray(dst_image, 'RGB'), ((col + 1) * w, 0))
    row_dlatents = np.stack([dst_dlatents[col]] * len(src_seeds))
    row_dlatents[:, style_ranges[col]] = src_dlatents[:, style_ranges[col]]
    row_images = Gs.components.synthesis.run(row_dlatents, randomize_noise=False, **synthesis_kwargs)
    for row, image in enumerate(list(row_images)):
        canvas.paste(PIL.Image.fromarray(image, 'RGB'), ((col + 1) * w, (row + 1) * h))
canvas.save(png)